# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [42]:
import pandas as pd

recipes = pd.read_csv('data/recipes_sample.csv')

random_recipes = recipes.sample(n=5)


id_width = random_recipes['id'].astype(str).str.len().max() + 8
minutes_width = len('minutes') + 4
if random_recipes['minutes'].astype(str).str.len().max() > minutes_width:
    minutes_width = random_recipes['minutes'].astype(str).str.len().max()


print(f"|{'id':^{id_width}}|{'minutes':^{minutes_width}}|")
print('|' + '-' * (id_width + minutes_width + 1) + '|')
for i in range(5):
    print(f"|{random_recipes['id'].iloc[[i]].to_string(index=False):^{id_width}}"
          f"|{random_recipes['minutes'].iloc[[i]].to_string(index=False):^{minutes_width}}|")

|      id      |  minutes  |
|--------------------------|
|     4035     |    30     |
|    378033    |    15     |
|    65729     |    65     |
|    390548    |    40     |
|    10456     |    80     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [90]:
from bs4 import BeautifulSoup


def show_info(recipe_id):
    recipes = pd.read_csv('data/recipes_sample.csv')
    with open('data/steps_sample.xml', 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    recipe_xml = soup.find('id', text=recipe_id).parent
    steps = recipe_xml.find_all('step')
    recipe = recipes[recipes['id'] == recipe_id].iloc[0]
    title = recipe['name']
    contributor_id = recipe['contributor_id']
    minutes = recipe['minutes']
    steps_list = [step.text for step in steps]
    output = f'"{title}"\n\n'
    output += '\n'.join([f"{i+1}. {step.text}" for i, step in enumerate(steps)])
    output += f"\n{'-'*10}\n"
    output += f"Автор: {contributor_id}\n"
    output += f"Среднее время приготовления: {minutes} минут."
    return output


print(show_info(170895))

"leeks and parsnips  sauteed or creamed"

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an addi

In [46]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

TypeError: show_info() got an unexpected keyword argument 'name'

In [ ]:
Я выполнил функцию через поиск информации по id рецепта, а не по вводу всех данных :(

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [111]:
import re


rexp = r'\d+\s(hours|hour|minutes|minute)'

with open('data/steps_sample.xml', 'r') as f:
    soup = BeautifulSoup(f, 'xml')
recipe_id = '25082'
recipe = soup.find('id', text=recipe_id).parent
steps = recipe.find_all('step')

for step in steps:
    match = re.search(rexp, step.text)
    if match:
        print(match.group(0))

20 minutes
10 minutes
2 hours
10 minutes
20 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [109]:
rexp = r'^this[\w\s]*,\s*but'

recipes['description'] = recipes['description'].fillna('')
matches = recipes['description'].str.contains(rexp)

print("Количество рецептов, содержащих шаблон: ", matches.sum())
print("\nПримеры описаний:")
for desc in recipes.loc[matches, 'description'].head(3):
    print('\n' + desc)

Количество рецептов, содержащих шаблон:  134

Примеры описаний:

this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.

this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

this is kind of similar to some of the other versions out there, but it is the best and easiest i have found


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [113]:
recipe = soup.find('id', text='72367').parent
steps = recipe.find_all('step')

for step in steps:
    step_text = step.text
    step_text = re.sub(r'\s*(\/)\s*', r'\1', step_text)
    print(step_text)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [117]:
import nltk
nltk.download('punkt')


steps = soup.find_all('step')
words = set()

for step in steps:
    text = step.text.lower()
    for word in nltk.word_tokenize(text):
        if word.isalpha():
            words.add(word)

print(len(words))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NfSix\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [119]:
recipes['sentences'] = recipes['description'].apply(nltk.sent_tokenize)

recipes['num_sentences'] = recipes['sentences'].apply(len)
top_5 = recipes.nlargest(5, 'num_sentences')


for i, row in top_5.iterrows():
    print(f"{row['name']} ({row['num_sentences']} предложений):\n{row['description']}\n")

my favorite buttercream icing for decorating (76 предложений):
this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.  it makes a delicious filling also between the layers of cakes and under fondant icing.  you can make roses but it takes 3 or more days to dry them depending on the humidity. 

there are many versions of “buttercream” icing. some are made with eggs and all butter.  some varieties, you have to cook your sugar to a softball stage.  others are 100% shortening or a combination of shortening and butter.

each decorator has his or her favorite.  i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch. but…. i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot days.  it melts!  a greasy puddle of melted icing on a cake plate is not something i want to look at or ea

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [125]:
nltk.download('averaged_perceptron_tagger')
def pos_tagging(sentence):
    tokens = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(tokens)
    result = ''
    for word, tag in pos_tags:
        result += f'{tag:^{len(word)}} '
    print(result)
    print(' '.join(sentence.split()))
    return

recipe_id = 241106
recipe_name = recipes.loc[recipes['id'] == recipe_id, 'name'].values[0]

pos_tagging(recipe_name)



   JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS   
eggplant steaks with chickpeas feta cheese and black olives


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\NfSix\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
